In [1]:
# coding: utf-8
from __future__ import print_function
import os
import numpy as np
import time
np.random.seed(1337)

import theano
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Flatten, Activation
from keras.layers import Convolution1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model
from keras.layers import Input, Dropout
from keras.optimizers import SGD, Adadelta
from keras.models import Sequential
import sys

BASE_DIR = '../..'
GLOVE_DIR = BASE_DIR + '/glove/'

TEXT_DATA_DIR = BASE_DIR + '/newsgroup/'

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
CONVOLUTION_FEATURE = 256
DENSE_FEATURE = 1024
DROP_OUT = 0.3

# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')
print('Embedding Dimesions: %s' % (str(EMBEDDING_DIM)))

embeddings_index = {}
fname = os.path.join(GLOVE_DIR, 'glove.6B.' + str(EMBEDDING_DIM) + 'd.txt')
f = open(fname)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                texts.append(f.read())
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Preparing embedding matrix.')

# prepare embedding matrix
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
# embedding_layer = Embedding(nb_words + 1,
#                             EMBEDDING_DIM,
#                             weights=[embedding_matrix],
#                             input_length=MAX_SEQUENCE_LENGTH,
#                             trainable=False)

print('Training model.')
model = Sequential()

model.add(Embedding(                          # Layer 0, Start
    input_dim=nb_words + 1,                   # Size to dictionary, has to be input + 1
    output_dim=EMBEDDING_DIM,                 # Dimensions to generate
    weights=[embedding_matrix],               # Initialize word weights
    input_length=MAX_SEQUENCE_LENGTH))        # Define length to input sequences in the first layer

model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(5))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print("Done compiling.")

Using gpu device 0: GeForce GTX 950 (CNMeM is disabled, cuDNN 5005)
Using Theano backend.


Indexing word vectors.
Embedding Dimesions: 100
Found 400000 word vectors.
Processing text dataset
Found 5000 texts.
Found 101187 unique tokens.
Shape of data tensor: (5000, 1000)
Shape of label tensor: (5000, 5)
Preparing embedding matrix.
Training model.
Done compiling.


In [2]:
start = time.time()

testX = x_val[:][0:4]
testY = y_val[:][0:4]
valX = x_val[:][4:]
valY = y_val[:][4:]

In [3]:
history = model.fit(x_train, y_train, validation_data=(valX, valY),
                    nb_epoch=10, batch_size=30)

print ("Training Time : ", time.time() - start)

Train on 4000 samples, validate on 996 samples
Epoch 1/10
4000/4000 [==============================] - 96s - loss: 1.5202 - acc: 0.2998 - val_loss: 1.3410 - val_acc: 0.3996

In [4]:
predY = np.array(model.predict(testX))

In [5]:
print(testY)

[[ 1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]]


In [6]:
print(predY)

[[  6.32459581e-01   6.13463158e-03   1.14259101e-03   1.66262104e-03
    4.95548779e-03]
 [  2.52683501e-04   1.22790569e-02   1.01056308e-01   1.94256834e-03
    3.62350052e-04]
 [  7.13797272e-05   1.51703600e-03   1.39352540e-02   2.99467742e-01
    9.86514729e-04]
 [  1.11907517e-04   2.63741046e-01   1.20613631e-02   1.95023153e-04
    3.62617546e-04]]


In [7]:
print('Shape of x_train tensor:', x_train.shape)
print('Shape of x_val tensor:', x_val.shape)
print('Shape of valX tensor:', valX.shape)
print('Shape of testX tensor:', testX.shape)

Shape of x_train tensor: (4000, 1000)
Shape of x_val tensor: (1000, 1000)
Shape of valX tensor: (996, 1000)
Shape of testX tensor: (4, 1000)


In [8]:
print(testX)

[[    0     0     0 ...,     7   401 13740]
 [    0     0     0 ...,   652  8513  6360]
 [    0     0     0 ..., 11268  3734   119]
 [    0     0     0 ...,   540   582   379]]


In [11]:
predmean = (predY[0][0] + predY[1][1] + predY[2][3] + predY[3][2]) / 4

In [13]:
print(predmean)

0.239066928625
